In [8]:
import openai
from anthropic import Anthropic
from typing import Optional, Literal

class LLMHandler:
    def __init__(self, api_key: Optional[str] = None, model_type: Literal["gpt4o", "claude"] = "gpt4o"):
        """
        LLM 핸들러 초기화
        Args:
            api_key: API 키 (없으면 환경변수에서 가져옴)
            model_type: 사용할 모델 타입 ("gpt4o" 또는 "claude")
        """
        self.model_type = model_type
        if model_type == "gpt4o":
            self.api_key = api_key or os.getenv("OPENAI_API_KEY")
            if not self.api_key:
                raise ValueError("OpenAI API 키가 필요합니다.")
            openai.api_key = self.api_key
        else:  # claude
            self.api_key = api_key or os.getenv("ANTHROPIC_API_KEY")
            if not self.api_key:
                raise ValueError("Anthropic API 키가 필요합니다.")
            self.client = Anthropic(api_key=self.api_key)

    def get_answer_by_prompting(self, prompt: str) -> str:
        """
        프롬프트를 받아 LLM의 응답을 반환
        Args:
            prompt: 입력 프롬프트
        Returns:
            LLM의 응답 텍스트
        """
        try:
            if self.model_type == "gpt4o":
                response = openai.ChatCompletion.create(
                    model="gpt-4o",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.7
                )
                return response.choices[0].message.content
            else:  # claude
                message = self.client.messages.create(
                    model="claude-3-5-sonnet-20241022",
                    max_tokens=1000,
                    temperature=0.7,
                    messages=[
                        {"role": "user", "content": prompt}
                    ]
                )
                return message.content[0].text

        except Exception as e:
            return f"Error: {str(e)}"

def main():
    # 모델 선택
    while True:
        model_choice = input("사용할 모델을 선택하세요 (1: GPT-4, 2: Claude): ").strip()
        if model_choice in ['1', '2']:
            break
        print("잘못된 입력입니다. 1 또는 2를 입력하세요.")
    
    model_type = "gpt4o" if model_choice == '1' else "claude"
    
    # API 키 입력
    api_key = input(f"{'OpenAI' if model_choice == '1' else 'Anthropic'} API 키를 입력하세요: ").strip()
    
    try:
        # LLM 핸들러 초기화
        handler = LLMHandler(api_key=api_key, model_type=model_type)
        
        # 대화 루프
        print("\n'quit' 또는 'exit'를 입력하면 프로그램이 종료됩니다.")
        while True:
            # 프롬프트 입력
            prompt = input("\n프롬프트를 입력하세요: ").strip()
            
            # 종료 조건
            if prompt.lower() in ['quit', 'exit']:
                print("프로그램을 종료합니다.")
                break
            
            # 빈 입력 처리
            if not prompt:
                print("프롬프트를 입력해주세요.")
                continue
            
            # 응답 받기
            print("\n응답을 기다리는 중...\n")
            response = handler.get_answer_by_prompting(prompt)
            print(f"[{model_type.upper()} 응답]:\n{response}\n")
            
    except Exception as e:
        print(f"오류가 발생했습니다: {str(e)}")

if __name__ == "__main__":
    main()


'quit' 또는 'exit'를 입력하면 프로그램이 종료됩니다.

응답을 기다리는 중...

[CLAUDE 응답]:
네, 저는 코딩과 프로그래밍에 대해 도움을 드릴 수 있습니다. 어떤 도움이 필요하신가요?

프로그램을 종료합니다.
